In [ ]:
# Setup
%pip install databricks-sdk==0.40.0 mlflow==2.18.0 --quiet
dbutils.library.restartPython()

In [ ]:
# Initialize
%run ../_resources/00-setup $reset_all_data=false

import sys
sys.path.append('../_resources')
from gamification_framework import (
    init_learner,
    display_challenge_intro,
    display_challenge_success
)

learner = init_learner()
display_challenge_intro(
    challenge_name="Multi-Agent Orchestration",
    difficulty="Advanced",
    points=400,
    description="Build a coordinated multi-agent system that handles complex, multi-step workflows through agent collaboration and intelligent routing."
)

---

## 📚 Pattern 1: Sequential Pipeline

The simplest multi-agent pattern: agents work in a chain.

```
Input → Agent 1 → Agent 2 → Agent 3 → Output
```

Each agent specializes in one step.

In [ ]:
from databricks.sdk import WorkspaceClient
import json

w = WorkspaceClient()

# Agent 1: Data Collector
data_collector_prompt = """
You are a Data Collection Specialist.
Your ONLY job is to extract and structure sensor data from text.

Input format: "Turbine [ID] shows vibration [VALUE] mm/s, temp [VALUE]°C, power [VALUE] MW"

Output format (JSON only, no other text):
{
  "turbine_id": "...",
  "vibration": float,
  "temperature": float,
  "power_output": float
}
"""

# Agent 2: Diagnostic Specialist  
diagnostic_prompt = """
You are a Turbine Diagnostic Specialist.
You receive structured sensor data and diagnose issues.

Input: JSON with sensor readings
Output: JSON with diagnosis:
{
  "issue_type": "bearing/gearbox/blade/electrical/sensor",
  "severity": "CRITICAL/HIGH/MEDIUM/LOW",
  "confidence": "percentage",
  "reasoning": "brief explanation"
}
"""

# Agent 3: Action Planner
action_planner_prompt = """
You are a Maintenance Action Planner.
You receive diagnostic data and create action plans.

Input: JSON with diagnosis
Output: JSON with action plan:
{
  "priority_score": 1-10,
  "immediate_actions": ["list of actions"],
  "required_resources": {"technicians": N, "parts": ["list"]},
  "estimated_cost": float,
  "estimated_downtime": "X hours"
}
"""

print("✅ Sequential pipeline agents defined")

In [ ]:
# Test the sequential pipeline

def run_sequential_pipeline(user_input: str):
    """Run three agents in sequence"""
    
    print("🔄 Starting Sequential Pipeline...\n")
    
    # Stage 1: Data Collection
    print("📊 Agent 1: Data Collector")
    response1 = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[
            {"role": "system", "content": data_collector_prompt},
            {"role": "user", "content": user_input}
        ],
        max_tokens=200
    )
    sensor_data = response1.choices[0].message.content
    print(f"Output: {sensor_data}\n")
    
    # Stage 2: Diagnosis
    print("🔍 Agent 2: Diagnostic Specialist")
    response2 = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[
            {"role": "system", "content": diagnostic_prompt},
            {"role": "user", "content": f"Analyze this data: {sensor_data}"}
        ],
        max_tokens=300
    )
    diagnosis = response2.choices[0].message.content
    print(f"Output: {diagnosis}\n")
    
    # Stage 3: Action Planning
    print("📋 Agent 3: Action Planner")
    response3 = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[
            {"role": "system", "content": action_planner_prompt},
            {"role": "user", "content": f"Create action plan for: {diagnosis}"}
        ],
        max_tokens=400
    )
    action_plan = response3.choices[0].message.content
    print(f"Output: {action_plan}\n")
    
    return {
        "sensor_data": sensor_data,
        "diagnosis": diagnosis,
        "action_plan": action_plan
    }

# Test it
test_input = "Turbine WT-042 shows vibration 15.2 mm/s, temp 98°C, power 1.2 MW"
result = run_sequential_pipeline(test_input)

print("="*70)
print("✅ Pipeline completed!")

---

## 📚 Pattern 2: Parallel Fan-Out

Multiple agents work simultaneously on different aspects:

```
         ┌→ Agent A ┐
Input ───┼→ Agent B ┼→ Aggregator → Output
         └→ Agent C ┘
```

Faster but requires result aggregation.

In [ ]:
# Define specialized parallel agents

safety_agent_prompt = """
You are a Safety Assessment Specialist.
Evaluate ONLY safety risks. Output JSON:
{
  "safety_risk": "CRITICAL/HIGH/MEDIUM/LOW/NONE",
  "hazards": ["list"],
  "safety_protocol": "required protocol"
}
"""

cost_agent_prompt = """
You are a Cost Analysis Specialist.
Evaluate ONLY financial impact. Output JSON:
{
  "revenue_loss_per_hour": float,
  "repair_cost_estimate": float,
  "total_financial_impact": float
}
"""

technical_agent_prompt = """
You are a Technical Feasibility Specialist.
Evaluate ONLY technical aspects. Output JSON:
{
  "parts_available": true/false,
  "expertise_required": "junior/mid/senior/expert",
  "repair_complexity": "simple/moderate/complex/expert-only",
  "estimated_hours": float
}
"""

print("✅ Parallel agents defined")

In [ ]:
# Run agents in parallel
import concurrent.futures
import time

def query_agent(prompt: str, user_message: str, agent_name: str):
    """Query a single agent"""
    start = time.time()
    response = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_message}
        ],
        max_tokens=300
    )
    elapsed = time.time() - start
    return {
        "agent": agent_name,
        "result": response.choices[0].message.content,
        "time_seconds": elapsed
    }

def run_parallel_fanout(issue_description: str):
    """Run multiple agents in parallel"""
    
    print("⚡ Starting Parallel Fan-Out...\n")
    start_time = time.time()
    
    agents = [
        (safety_agent_prompt, issue_description, "Safety Agent"),
        (cost_agent_prompt, issue_description, "Cost Agent"),
        (technical_agent_prompt, issue_description, "Technical Agent")
    ]
    
    # Execute in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(query_agent, *agent) for agent in agents]
        results = [f.result() for f in concurrent.futures.as_completed(futures)]
    
    total_time = time.time() - start_time
    
    # Display results
    for result in results:
        print(f"\n{result['agent']} ({result['time_seconds']:.2f}s):")
        print(result['result'])
        print("-" * 60)
    
    print(f"\n⚡ Total parallel execution time: {total_time:.2f}s")
    print(f"   (vs ~{sum(r['time_seconds'] for r in results):.2f}s sequential)")
    
    return results

# Test it
test_issue = "Gearbox failure in Turbine WT-042, high vibration and temperature, currently generating 50% capacity"
parallel_results = run_parallel_fanout(test_issue)

---

## 🎯 Challenge: Build a Hierarchical Multi-Agent System

**Your Task:** Create a 3-tier agent hierarchy:

```
                    Supervisor Agent
                          |
        ┌─────────────────┼─────────────────┐
        ↓                 ↓                 ↓
   Diagnostic Team   Resource Team    Communication Team
        |
   ┌────┼────┐
   ↓    ↓    ↓
  Sensor Analysis
  Error Code Analysis  
  Historical Pattern Analysis
```

**Requirements:**

1. **Supervisor Agent**: Routes tasks to appropriate teams, aggregates results
2. **Team Agents**: Coordinate specialized sub-agents
3. **Specialist Agents**: Handle specific technical tasks
4. **Communication Protocol**: Define how agents share information
5. **Error Handling**: Gracefully handle agent failures

In [ ]:
# 💪 YOUR TURN - Build the hierarchical system

# Step 1: Define your supervisor agent
supervisor_prompt = """
# TODO: Create a supervisor agent that:
# - Analyzes incoming requests
# - Determines which teams to engage
# - Coordinates their work
# - Aggregates results into a final decision
# 
# You are the Turbine Emergency Response Supervisor...
"""

# Step 2: Define your team coordinators
diagnostic_team_prompt = """
# TODO: Create diagnostic team coordinator
# Manages: sensor analysis, error codes, historical patterns
"""

resource_team_prompt = """
# TODO: Create resource team coordinator  
# Manages: technician availability, parts inventory, budget
"""

communication_team_prompt = """
# TODO: Create communication team coordinator
# Manages: stakeholder updates, status reports, escalations
"""

# Step 3: Define specialist agents (at least 3)
# TODO: Create sensor_analysis_agent_prompt
# TODO: Create error_code_agent_prompt
# TODO: Create historical_pattern_agent_prompt

print("✅ Agent hierarchy defined")

In [ ]:
# Step 4: Implement the orchestration logic

class MultiAgentOrchestrator:
    """Manages hierarchical multi-agent execution"""
    
    def __init__(self):
        self.w = WorkspaceClient()
        self.conversation_history = []
    
    def query_agent(self, agent_prompt: str, message: str, agent_name: str):
        """Query a single agent"""
        # TODO: Implement agent querying with error handling
        pass
    
    def run_supervisor(self, emergency_report: str):
        """Supervisor analyzes and delegates"""
        # TODO: Implement supervisor logic
        # 1. Analyze the emergency
        # 2. Decide which teams to activate
        # 3. Return delegation plan
        pass
    
    def run_team(self, team_prompt: str, task: str, team_name: str):
        """Team coordinator manages specialists"""
        # TODO: Implement team coordination
        # 1. Break down task for specialists
        # 2. Collect specialist results
        # 3. Synthesize team report
        pass
    
    def run_full_hierarchy(self, emergency_report: str):
        """Execute complete hierarchical workflow"""
        # TODO: Orchestrate the full workflow:
        # 1. Supervisor analyzes and delegates
        # 2. Teams execute tasks in parallel
        # 3. Supervisor aggregates results
        # 4. Return final emergency response plan
        
        print("🏗️ Starting Hierarchical Multi-Agent System...\n")
        
        # Your implementation here
        
        return {"status": "TODO: Implement"}

# Test your orchestrator
orchestrator = MultiAgentOrchestrator()

emergency = """
EMERGENCY ALERT - Storm Damage Assessment
- 5 turbines offline (WT-042, WT-043, WT-044, WT-045, WT-046)
- WT-042: Gearbox failure (critical)
- WT-043: Blade damage (high)
- WT-044: Sensor malfunction (medium)
- WT-045: Power electronics failure (high)
- WT-046: Emergency shutdown - unknown cause (critical)

Available resources:
- 3 senior technicians
- 2 junior technicians  
- Limited replacement parts inventory
- Emergency budget: $500,000

Required: Prioritized action plan within 15 minutes
"""

result = orchestrator.run_full_hierarchy(emergency)
print(json.dumps(result, indent=2))

---

## 📊 Evaluation Criteria

Your multi-agent system will be evaluated on:

1. **Correctness** (40 pts): Does it produce a valid emergency response plan?
2. **Efficiency** (30 pts): Does it use parallel execution where beneficial?
3. **Coordination** (30 pts): Do agents properly share information?
4. **Robustness** (25 pts): Does it handle errors gracefully?
5. **Scalability** (25 pts): Could it handle more agents/tasks?

**Total: 150 points** (bonus points available)

### Self-Assessment Checklist

- [ ] Supervisor correctly delegates to teams
- [ ] Teams coordinate multiple specialists
- [ ] Results are aggregated coherently
- [ ] Error handling prevents system crashes
- [ ] Response time is reasonable (<30s total)
- [ ] Output is actionable and prioritized

In [ ]:
# Automated evaluation (basic checks)

def evaluate_multi_agent_system(orchestrator, test_emergency):
    """Evaluate the multi-agent system"""
    score = 0
    feedback = []
    
    try:
        import time
        start = time.time()
        result = orchestrator.run_full_hierarchy(test_emergency)
        execution_time = time.time() - start
        
        # Check if result is structured
        if isinstance(result, dict) and len(result) > 0:
            score += 40
            feedback.append("✅ System produces structured output (+40)")
        
        # Check execution time (should benefit from parallelization)
        if execution_time < 30:
            score += 30
            feedback.append(f"✅ Efficient execution: {execution_time:.2f}s (+30)")
        elif execution_time < 60:
            score += 15
            feedback.append(f"⚠️ Moderate efficiency: {execution_time:.2f}s (+15)")
        
        # Check if multiple agents were used
        if len(orchestrator.conversation_history) >= 3:
            score += 30
            feedback.append("✅ Multiple agents coordinated (+30)")
        
        feedback.append(f"\n📊 Final Score: {score}/100")
        
        if score >= 80:
            feedback.append("\n🎉 Excellent work! Badge earned!")
            return score, feedback, True
        else:
            feedback.append("\n💡 Keep refining your system!")
            return score, feedback, False
            
    except Exception as e:
        feedback.append(f"❌ System error: {str(e)}")
        return 0, feedback, False

# Run evaluation
score, feedback, badge_earned = evaluate_multi_agent_system(orchestrator, emergency)

for item in feedback:
    print(item)

if badge_earned:
    learner.complete_challenge("multi_agent_orchestration", points=400)
    learner.award_badge("multi_agent_architect")
    display_challenge_success("Multi-Agent Orchestration", 400)

---

## 🎓 Key Takeaways

You've learned:

✅ **Sequential Pipelines** - Simple linear agent workflows  
✅ **Parallel Fan-Out** - Concurrent agent execution  
✅ **Hierarchical Architectures** - Multi-tier agent coordination  
✅ **State Management** - Sharing context between agents  
✅ **Performance Optimization** - Balancing speed and complexity

### 🏆 When to Use Each Pattern

| Pattern | Best For | Pros | Cons |
|---------|----------|------|------|
| Sequential | Simple workflows, dependencies | Easy to debug | Slower |
| Parallel | Independent tasks | Fastest | Complex aggregation |
| Hierarchical | Complex coordination | Scalable, organized | Most complex |

---

## 🚀 Next Steps

You're now ready for the ultimate challenges:

- **05.X-real-world-scenarios**: Apply your multi-agent system to emergencies
- **05.Y-performance-optimization**: Optimize latency and costs
- **05.Z-agent-evaluation**: Build comprehensive evaluation frameworks

In [ ]:
# Check your overall progress
learner.display_progress()